In [1]:
import os, sys
import os, sys
project_dir = os.getcwd()
if project_dir not in sys.path:
    sys.path.append(project_dir)

import numpy as np
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
from spectral.algorithms import spectral_angles
from dataset import DermaDataset

In [2]:
dataset_root_dir = "/home/abian/Data/Dataset/IUMA/DermaDatabase/dataCubes/"
train_dir = ['train', 'validation']
dataset_dir = list(map(lambda x: os.path.join(dataset_root_dir, x), train_dir))

dataset = DermaDataset(dataset_dir)
x, y = dataset.get()

# Data Preprocessing

## Calibration 

$\begin{align}
    C_i = 100 * \frac{R_i - D_r}{W_r - D_r}
\end{align}$

where $C_i$ is the calibrated image, $R_i$ note raw image and the $W_r$ and $D_r$ represents the white and dark reference image, respectively.


In [3]:
# W_r y D_r es una imagen
def calibrate(img, w_r, d_r):
    if not(w_r.shape == d_r.shape == img.shape):
        assert('Dimensionality error')
    
    return 100  * (img - d_r) / (w_r - d_r)

# Normalize data
$\begin{align}
    P'_i = \frac{P_i - P_{min}}{P_{max} - P_{min}}
\end{align}$

where $P'_i$ is the normalized pixel value, $P_i$ the reflectance of the pixel, $P_{min}$ and $P_{max}$ is the minimum and maximum reflectance value, respectively.

In [4]:
def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

# Sampling Interval Analysis

Selected equidistant band ...

$\begin{align}
    Sampling Interval (nm) = \frac{\lambda_{max} - \lambda_{min}}{N_{\lambda}}
\end{align}$

where $\lambda_{max} - \lambda_{min}$ is the difference between the mamum and minimum wavelength and $N_{\lambda}$ is the number of band captured by the sensor.

In [5]:
def sampling_interval(lambda_min, lambda_max, n_spectral_bands):
    '''
        Param:
        -----
            lambda_min (int): minimum wavelenght
            lambda_max (int): maximum wavelenght
            n_spectral_bands (int): number of spectral bands captured by the sensor
    '''
    return (lambda_max - lambda_min) / n_spectral_bands

# Dataset reduction

**Spectral Angle Mapper**

$\alpha = cos^{-1}\left ( \frac{\sum_{i = 1}^{nb} t_{i} r_{i}}{(\sum_{i = 1}^{nb} t_{i}^2)^{\frac{1}{2}} (\sum_{i = 1}^{nb} r_{i}^2)^{\frac{1}{2}}} \right )$

where

* $\alpha$ = spectral angle between the standard and the spectral curve of the pixel
* $nb$ = number of spectral channels
* $t$ = vector of spectral response of the standard
* $r$ = the spectral response vector of the analyzed pixel

In [6]:
from spectral.algorithms import spectral_angles
from sklearn.cluster import KMeans

def spectral_angles_pixel(x, ref):
    '''
        For pixel input, the original function is prepare for image
    '''
    return spectral_angles(x[np.newaxis,:], ref)[0]

def get_most_relevant_samples(x, centroid, n_samples_per_centroid=10):
    '''
        Paper: Most Relevant Spectral Bands Identification for Brain
        Cancer Detection Using Hyperspectral Imaging    
    '''
    if len(x.shape) != 2:
        assert 'X shape error!'
    
    output = None
    result = spectral_angles_pixel(x, centroid)
    for i in range(len(centroid)):       
        ind = np.argpartition(result[i], -n_samples_per_centroid)[-n_samples_per_centroid:]
        if i == 0:
            output = x[ind]
        else:
            output = np.concatenate([output, x[ind]], axis=0)

    return output

def get_most_relevant_samples_idx(x, centroid, n_samples_per_centroid=10):
    '''
        Paper: Most Relevant Spectral Bands Identification for Brain
        Cancer Detection Using Hyperspectral Imaging    
    '''
    if len(x.shape) != 2:
        assert 'X shape error!'
    
    output = np.array([], dtype=np.uint)
    result = spectral_angles_pixel(x, centroid)
    print(result.shape)
    for i in range(len(centroid)):       
        ind = np.argpartition(result[:, i], -n_samples_per_centroid)[-n_samples_per_centroid:].astype(np.uint)
        output = np.concatenate((output, ind))

    return output

def dataset_reduction(x, y, n_centroid_per_class=100, random_state=123):
    class_label = np.unique(y)
    final_x = None
    final_y = None
    final_idx = np.array([])
    for i in range(len(class_label)):
        print('CLass: {}'.format(i))
        idx = np.where(y==class_label[i])
        _x = x[idx]
        # print(_x)
        if len(idx[0]) > 1000:
            kmeans = KMeans(n_clusters=n_centroid_per_class, random_state=random_state).fit(_x)
            centroid = kmeans.cluster_centers_
            # _x = get_most_relevant_samples_idx(_x, centroid)
            most_relevant_samples_idx = get_most_relevant_samples_idx(_x, centroid)
            # print(idx[0][most_relevant_samples_idx])
            final_idx = np.concatenate((final_idx, idx[0][most_relevant_samples_idx]))
            # test = np.concatenate([test, get_most_relevant_samples_idx(_x, centroid)]) 
            # print(test)

    #     if i == 0:
    #         final_x = _x
    #         final_y = np.full((_x.shape[0],), class_label[i])
    #     else:
    #         final_x = np.concatenate([final_x, _x], axis=0)
    #         final_y = np.concatenate([final_y, np.full((_x.shape[0],), class_label[i])], axis=0)
    
    return final_idx
    # return final_x, final_y

In [7]:
class_label = np.unique(y)
for i in range(len(class_label)):
    class_idx = np.argwhere(y==class_label[i]).flatten()
    _x = x[class_idx]
    kmeans = KMeans(n_clusters=100, random_state=123).fit(_x)
    centroid = kmeans.cluster_centers_
    labels = kmeans.labels_
    print(labels.shape)
    # result = spectral_angles_pixel(x, centroid)
    # output = np.array([], dtype=np.uint)
    # for i in range(len(centroid)):       
    #     ind = np.argpartition(result[:, i], -10)[-10:].astype(np.uint)
    #     output = np.concatenate((output, ind))
    # break

(3343,)
(7295,)


In [62]:
idx = np.argwhere(y==class_label[i]).flatten()
print(idx)

[  100   101   102 ... 10635 10636 10637]


In [121]:
def get_most_relevant_samples_idx(x, labels, centroids, n_samples_per_centroid=10):
    '''
        Returns the index of samples where 

        Paper: Most Relevant Spectral Bands Identification for Brain
        Cancer Detection Using Hyperspectral Imaging    

        Params:
        ------
            x (np.ndarray): Data
            labels (np.ndarray): Cluster label of x
            centroids (np.ndarray): The centroids from clustering algorithm
            n_samples_per_centroid: int, 10
    '''
    if len(x.shape) != 2:
        assert 'X shape error!'

    if x.shape != labels.shape:
        assert 'x and labels do not match in dimension'

    if np.unique(labels).size != centroids.shape[0]:
        assert 'The number of labels and centroids does not match'
    
    sam_result = spectral_angles_pixel(x, centroids)
    selected_samples_idx = np.array([], dtype=np.uint)

    for label in np.unique(labels):
        cluster_samples_idx = np.argwhere(labels==label).flatten()
        n_samples_per_centroid = n_samples_per_centroid if cluster_samples_idx.size >= n_samples_per_centroid else cluster_samples_idx.size
        ind = np.argpartition(sam_result[cluster_samples_idx, label], -n_samples_per_centroid)[-n_samples_per_centroid:].astype(np.uint)
        selected_samples_idx = np.concatenate((selected_samples_idx, cluster_samples_idx[ind])).astype(np.uint)

    return selected_samples_idx

In [122]:
n_samples_per_centroid=20
random_state=123

def dataset_reduction(x, y, n_clusters=100, random_state=123):
    class_label = np.unique(y)
    selected_samples_idx = np.array([], dtype=np.uint)
    for i in class_label:
        class_samples_idx = np.argwhere(y==i).flatten()
        _x = x[class_samples_idx]
        kmeans = KMeans(n_clusters=n_clusters, random_state=random_state).fit(_x)
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_
        
        selected_samples_class_idx = get_most_relevant_samples_idx(_x, labels, centroids, n_samples_per_centroid)
        selected_samples_idx = np.concatenate((selected_samples_idx, class_samples_idx[selected_samples_class_idx])).astype(np.uint)

    return selected_samples_idx

In [119]:
print(selected_samples_idx.shape)
test = y[selected_samples_idx]
print(test[test==0].shape)
# print(selected_samples_idx.shape)

(2000,)
(1000,)


In [89]:
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int32)

In [120]:
test = dataset_reduction(x, y)

0
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
(100, 116)
[301 261 262 ... 621 529 677]
1
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
(100, 116)
[2326 2755 2396 ...  867  581  420]


In [105]:
class_label = np.unique(y)
for i in class_label:
    class_idx = np.argwhere(y==i).flatten()
    _x = x[class_idx]
    kmeans = KMeans(n_clusters=100, random_state=123).fit(_x)
    centroid = kmeans.cluster_centers_
    labels = kmeans.labels_
    break

In [106]:
print(centroid.shape)

(100, 116)


In [124]:
n_samples_per_centroid = 10
# mapper_result = spectral_angles_pixel(_x, centroid)
# selected_samples_idx = np.array([], dtype=np.uint)

result = get_most_relevant_samples_idx(_x, labels, centroid, n_samples_per_centroid=n_samples_per_centroid)
# print(result)
print(np.unique(result).size == result.size)

True


In [32]:
n_samples_per_centroid = 10
mapper_result = spectral_angles_pixel(_x, centroid)
selected_samples_idx = np.array([], dtype=np.uint)

for label in np.unique(labels):
    samples_idx = np.argwhere(labels==label).flatten()
    ind = np.argpartition(mapper_result[samples_idx, i], -n_samples_per_centroid)[-n_samples_per_centroid:].astype(np.uint)
    selected_samples_idx = np.concatenate((selected_samples_idx, samples_idx[ind])).astype(np.uint)
    # print(selected_samples_idx)
    # print(samples_idx[ind])
    # if label == 2:
    #     break
# labels[labels==2].shape

In [36]:
print(selected_samples_idx.shape)
print(np.unique(selected_samples_idx).shape)

(1000,)
(1000,)


In [ ]:
_x[samples_idx]

In [ ]:
np.unique(output)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()

for i in range(c.shape[0]):
    plt.plot(c[i])

plt.grid()

In [ ]:
x.reshape(1, -1).shape

In [ ]:
print(output.shape)
print(np.unique(output).shape)

In [ ]:
a = np.array([])
for i in range(100):
    min_value = np.min(result[:, i])
    a = np.concatenate([a, np.argwhere(result[:, i] == min_value).reshape(-1)])

np.unique(a)


In [ ]:
a.shape

In [ ]:
# x_red, y_red = dataset_reduction(x, y, n_centroid_per_class=100)
samples_idx = dataset_reduction(x, y, n_centroid_per_class=100)
samples_idx
print(np.unique(samples_idx))
# print(x_red.shape)
# print(y_red.shape)

In [ ]:
test = y[samples_idx.astype(np.uint)]
len(test[test==1])

In [ ]:
test = np.array([])
test.size

In [137]:
BaseUnderSampler._sampling_strategy_docstring

"sampling_strategy : float, str, dict, callable, default='auto'\n        Sampling information to sample the data set.\n\n        - When ``float``, it corresponds to the desired ratio of the number of\n          samples in the minority class over the number of samples in the\n          majority class after resampling. Therefore, the ratio is expressed as\n          :math:`\\alpha_{us} = N_{m} / N_{rM}` where :math:`N_{m}` is the\n          number of samples in the minority class and\n          :math:`N_{rM}` is the number of samples in the majority class\n          after resampling.\n\n          .. warning::\n             ``float`` is only available for **binary** classification. An\n             error is raised for multi-class classification.\n\n        - When ``str``, specify the class targeted by the resampling. The\n          number of samples in the different classes will be equalized.\n          Possible choices are:\n\n            ``'majority'``: resample only the majority class;

In [163]:
from imblearn.under_sampling.base import BaseUnderSampler
from imblearn.utils import Substitution
from imblearn.utils._docstring import _n_jobs_docstring
from spectral.algorithms import spectral_angles
from sklearn.cluster import KMeans
from sklearn.utils import _safe_indexing


# @Substitution(
#     sampling_strategy=BaseUnderSampler._sampling_strategy_docstring,
#     n_jobs=_n_jobs_docstring,
# )
## Por ahora no usar!!! Está en pruebas!!

class HyperSpectralUnderSampler(BaseUnderSampler):
    ''' 
        Class to perform HyperSpectral data under-sampling.

        Under-sample the different class(es) by K-Mean unsupervised clustering approach. The K-Means clustering 
        is applied independently to each group of labeled pixels in order to obtain K clusters per group. In order 
        to reduce the original training dataset, such centroids are employed to identify the most representative pixels of
        each class by using the Spectral Angle [2] algorithm. For each cluster centroid, only the S most similar
        samples are selected.

        Parameters
        ----------
        n_clusters: int, default=100
            The number of centroids used in K-Mean clustering (K).
        
        samples_per_class: int, default=10
            The number of most similiar signals to select (S)

        {random_state}

        References
        ----------
          [1] Martinez, B., Leon, R., Fabelo, H., Ortega, S., Piñeiro, J. F., Szolna, A., ... & M Callico, G. (2019). Most
          relevant spectral bands identification for brain cancer detection using hyperspectral imaging.Sensors, 19(24), 5481.
          
          [2] Rashmi, S.; Addamani, S.; Ravikiran, A. Spectral Angle Mapper algorithm for remote sensing image classification. 
          IJISET Int. J. Innov. Sci. Eng. Technol. 2014, 1, 201–20
    '''
    def __init__(self, *, sampling_strategy="auto", n_clusters=100, samples_per_cluster=10, random_state=None):
        super().__init__(sampling_strategy=sampling_strategy)
        self.K = n_clusters
        self.S = samples_per_cluster
        self.random_state = random_state

    def _spectral_angles_pixel(self, x, ref):
        '''
            For pixel input, the original function is prepare for image
        '''
        return spectral_angles(x[np.newaxis,:], ref)[0]

    def get_most_relevant_samples_idx(self, x, labels, centroids, n_samples_per_centroid=10):
        if len(x.shape) != 2:
            assert 'X shape error!'

        if x.shape != labels.shape:
            assert 'x and labels do not match in dimension'

        if np.unique(labels).size != centroids.shape[0]:
            assert 'The number of labels and centroids does not match'
        
        sam_result = self._spectral_angles_pixel(x, centroids)
        selected_samples_idx = np.array([], dtype=np.uint)

        for label in np.unique(labels):
            cluster_samples_idx = np.argwhere(labels==label).flatten()
            n_samples_per_centroid = n_samples_per_centroid if cluster_samples_idx.size >= n_samples_per_centroid else cluster_samples_idx.size
            ind = np.argpartition(sam_result[cluster_samples_idx, label], -n_samples_per_centroid)[-n_samples_per_centroid:].astype(np.uint)
            selected_samples_idx = np.concatenate((selected_samples_idx, cluster_samples_idx[ind])).astype(np.uint)

        return selected_samples_idx

    def _fit_resample(self, X, y):
        class_label = np.unique(y)
        selected_samples_idx = np.array([], dtype=np.uint)
        for i in class_label:
            class_samples_idx = np.argwhere(y==i).flatten()
            _x = x[class_samples_idx]
            kmeans = KMeans(n_clusters=self.K, random_state=self.random_state).fit(_x)
            labels = kmeans.labels_
            centroids = kmeans.cluster_centers_

            selected_samples_class_idx = self.get_most_relevant_samples_idx(_x, labels, centroids, self.S)
            selected_samples_idx = np.concatenate((selected_samples_idx, class_samples_idx[selected_samples_class_idx])).astype(np.uint)

        self.sample_indices_ = selected_samples_idx

        return _safe_indexing(X, self.sample_indices_), _safe_indexing(y, self.sample_indices_)


In [174]:
sampling_strategy = {0: 10, 1: 20}
a = HyperSpectralUnderSampler(sampling_strategy=sampling_strategy, n_clusters=50, samples_per_cluster=10, random_state=123)
test_x, test_y = a.fit_resample(x,y)

In [176]:
for target_class in a.sampling_strategy_.keys():
    n_samples = a.sampling_strategy_[target_class]
    target_class_indices = np.flatnonzero(y == target_class)

    print(n_samples, target_class_indices.shape)

10 (3343,)
20 (7295,)


In [165]:
# test_x.shape
print(test_y.shape)
print(test_y[test_y==0].shape)

(1000,)
(500,)


In [ ]:
x_red, y_red = dataset_reduction(x, y)
print(x_red.shape)
print(y_red.shape)

# Optimization

## Steps involved in HyperOptimization using Scikit-Optimizer

1. Define the space of hyperparameters to search
1. Define the function used to evaluate a given configuration
1. Minimize the loss using Space and Function defined in previous steps.

In [ ]:
import skopt

from feature_selection import FeatureSelection, FeatureEquidistantSelection
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Integer


# pipe = Pipeline([("transform", FeatureEquidistantSelection()), ('svc', SVC())])
pipe = Pipeline([("transform", FeatureSelection()), ('svc', SVC())])

params = dict()
n_features = x.shape[1]
# params['transform__n_features_to_select'] = (8, 34, 'uniform')
params['transform__selected_features'] = Integer(1, float(2**(116)-1), 'log-uniform')
params['svc__C'] = (1e-6, 100.0, 'log-uniform')
params['svc__gamma'] = (1e-6, 100.0, 'log-uniform')
params['svc__degree'] = (1,5)
params['svc__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = BayesSearchCV(estimator=pipe, search_spaces=params, n_jobs=-1, cv=cv)
# perform the search
search.fit(x, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

# Ant Colony Optimization (Testing)

**Buff, no furula. Buscar otra lib?**

probar **scikit-opt**: https://github.com/guofei9987/scikit-opt

https://www.youtube.com/watch?v=YFN_fJEu63w

In [ ]:
nodes = []
for _ in range(20):
  x = np.random.uniform(-10, 10)
  y = np.random.uniform(-10, 10)
  nodes.append((x, y))

nodes

In [ ]:
import pants
import math

def euclidean(a, b):
    return math.sqrt(pow(a[1] - b[1], 2) + pow(a[0] - b[0], 2))

In [ ]:
world = pants.World(nodes, euclidean)
solver = pants.Solver()
# solution = solver.solve(world)

solutions = solver.solutions(world)


In [ ]:
# print(solution.distance)
# print(solution.tour)    # Nodes visited in order
# print(len(solution.tour))
# print(solution.path)    # Edges taken in order
# print(len(solution.path))

best = float("inf")
for solution in solutions:
    assert solution.distance < best
    best = solution.distance
    print(best)
    print(len(solution.path))

print(best)

For each centroid, only the 10 most similar pixels are selected, having a total of 1000 pixels per class (100 centroids ×10 pixels). Thus, the reduced dataset is
intended to avoid the inclusion of redundant information in the training of the supervised classifier...

In [ ]:
solver.trace_elite[0]

In [ ]:
n_cluster = 2

X = np.array([[1, 2.5], [1, 4.1], [1, 0.1],
        [10, 2.1], [10, 4.9], [10, 0]])

kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(X)
centroid = kmeans.cluster_centers_
for x in X:
    plt.scatter(x=x[0], y=x[1], alpha=.2)

for x in centroid:
    plt.scatter(x=x[0], y=x[1], marker='*')

plt.show()